In [1]:
import os
# import cProfile
# import pstats
# from prometheus_client import start_http_server, Summary, Gauge
# import pdb #import for debugging


In [2]:
%pwd

'c:\\Users\\etrou\\OneDrive\\Desktop\\SE489GroupProjectGit\\group_project_se489\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\etrou\\OneDrive\\Desktop\\SE489GroupProjectGit\\group_project_se489'

In [5]:
import gdown
from se489_group_project import logger

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/eTroupe5201/GroupProjectSE489.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="eTroupe5201"
os.environ["MLFLOW_TRACKING_PASSWORD"]="635dd059be61c5cb0404b966309b062aa641c766"

In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("data/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from se489_group_project.constants import *
from se489_group_project.utility.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.data_storage])

    
    def get_evaluation_config(self) -> ModelEvaluationConfig:
        eval_config = ModelEvaluationConfig(
            path_of_model="data/training/model.h5",
            training_data="data/raw/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/eTroupe5201/GroupProjectSE489.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [16]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, log_loss, balanced_accuracy_score, matthews_corrcoef

In [13]:
class Evaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        # # Initialize Prometheus metrics
        # self.evaluation_time = Summary('evaluation_processing_seconds', 'Time spent evaluating the model')
        # self.model_loss = Gauge('model_loss', 'Model evaluation loss')
        # self.model_accuracy = Gauge('model_accuracy', 'Model evaluation accuracy')

        # # Start Prometheus HTTP server
        # start_http_server(8000)

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def calculate_metrics(self):
        y_true = self.valid_generator.classes
        y_pred = self.model.predict(self.valid_generator)
        y_pred_classes = y_pred.argmax(axis=-1)

        metrics = {
            'f1_score': f1_score(y_true, y_pred_classes, average='weighted'),
            'precision_score': precision_score(y_true, y_pred_classes, average='weighted'),
            'recall_score': recall_score(y_true, y_pred_classes, average='weighted'),
            'accuracy': accuracy_score(y_true, y_pred_classes),
            'roc_auc_score': roc_auc_score(y_true, y_pred_classes, multi_class='ovr'),
            'log_loss': log_loss(y_true, y_pred),
            'balanced_accuracy': balanced_accuracy_score(y_true, y_pred_classes),
            'matthews_corrcoef': matthews_corrcoef(y_true, y_pred_classes)
        }
        return metrics

    def evaluation(self):
        #breakpoint for debugging
        #pdb.set_trace()
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.metrics = self.calculate_metrics()
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Log additional metrics
            mlflow.log_metrics(metrics)
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [14]:
import cProfile
import pstats

def analyze(file):
    p = pstats.Stats(file)
    # Print top 10 functions sorted by cumulative time
    print("\nTop 10 functions sorted by cumulative time:")
    p.sort_stats('cumtime').print_stats(10)
    
    # Print top 10 functions sorted by total time
    print("\nTop 10 functions sorted by total time:")
    p.sort_stats('tottime').print_stats(10)

In [15]:
import subprocess
def main():
    try:
        log_dir = os.path.join(os.getcwd(), "se489_group_project", "visualizations")
        file = os.path.join(log_dir, 'cprofile_stats_model_evaluation.prof')
        config = ConfigurationManager()

        eval_config = config.get_evaluation_config()
        evaluation = Evaluation(eval_config)
        profiler = cProfile.Profile()
        profiler.enable()
        evaluation.evaluation()
        evaluation.log_into_mlflow()
        profiler.disable()
        profiler.dump_stats(file)
        profile_file_full_path = os.path.abspath(file)
        analyze(
            profile_file_full_path)
        #Automatically open snakeviz to visualize the profiling results
        try:
            subprocess.Popen(["snakeviz", profile_file_full_path])
        except FileNotFoundError:
            print("snakeviz is not installed or not found in the system path.")

    except Exception as e:
        raise e
if __name__ == "__main__":
    main()

[2024-05-30 22:41:50,031: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-30 22:41:50,033: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-30 22:41:50,033: INFO: common: created directory at: data]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 9s 912ms/step - loss: 0.6620 - accuracy: 0.5755
[2024-05-30 22:41:59,181: INFO: common: json file saved at: scores.json]


2024/05/30 22:42:00 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-05-30 22:42:03,375: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2024-05-30 22:42:04,489: INFO: builder_impl: Assets written to: C:\Users\etrou\AppData\Local\Temp\tmphlcpxa_j\model\data\model\assets]


c:\Users\etrou\anaconda3\envs\grp_env\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/05/30 22:42:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 22
Created version '22' of model 'VGG16Model'.



Top 10 functions sorted by cumulative time:
Thu May 30 22:42:56 2024    c:\Users\etrou\OneDrive\Desktop\SE489GroupProjectGit\group_project_se489\se489_group_project\visualizations\cprofile_stats_model_evaluation.prof

         6410260 function calls (6250225 primitive calls) in 66.884 seconds

   Ordered by: cumulative time
   List reduced from 5716 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   57.741   57.741 C:\Users\etrou\AppData\Local\Temp\ipykernel_31112\3041503622.py:56(log_into_mlflow)
        1    0.000    0.000   56.410   56.410 c:\Users\etrou\anaconda3\envs\grp_env\lib\site-packages\mlflow\tensorflow\__init__.py:124(log_model)
        1    0.000    0.000   56.410   56.410 c:\Users\etrou\anaconda3\envs\grp_env\lib\site-packages\mlflow\models\model.py:492(log)
       20    0.001    0.000   45.497    2.275 c:\Users\etrou\anaconda3\envs\grp_env\lib\site-packages\mlflow\utils\rest_utils.py:101(